## Dataset Details:
https://www.kaggle.com/competitions/titanic/data

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [13]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

# Function returns RandomForestRegressor estimates
def predict_RFR(X_train, y_train, X_valid, n_estimators=10): 
    model = RandomForestRegressor(n_estimators=n_estimators, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return preds

In [28]:
# Function returns list of cols containing a "null"
def null_cols(X_train):
  cols_with_missing = [col for col in X_train.columns if X_train[col].isnull().any()]
  return cols_with_missing

# Function returns list of categorical columns
def cat_cols(X_train):
  s = (X_train.dtypes == 'object')
  return list(s[s].index)

In [31]:
from sklearn.preprocessing import OrdinalEncoder

# Function for ordinal encoding
def ordinal_enc(X_train, X_valid, object_cols):
  # Make copy to avoid changing original data 
  label_X_train = X_train.copy()
  label_X_valid = X_valid.copy()

  # Apply ordinal encoder to each column with categorical data
  ordinal_encoder = OrdinalEncoder()
  label_X_train[object_cols] = ordinal_encoder.fit_transform(X_train[object_cols])
  label_X_valid[object_cols] = ordinal_encoder.transform(X_valid[object_cols])
  return [label_X_train, label_X_valid]

In [3]:
train_df = pd.read_csv("./data/train.csv")
test_df = pd.read_csv("./data/test.csv")
testv_df = pd.read_csv("./data/gender_submission.csv")

In [5]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
train_df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [8]:
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [9]:
testv_df.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [10]:
X_train = train_df.copy().drop(["Survived"], axis=1)
X_valid = test_df.copy()

y_train = train_df.copy()['Survived']
y_valid = testv_df.copy()['Survived']

#### Cleaning Data

In [16]:
X_train.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [34]:
# original approach:
# 1. Ordinal Encoding 'Sex', 1-> male; 0-> female
# 2. Drop other categorical features, looks redundant, and null_cols
# 3. Drop PassengerId, SibSp -> improves MAE scores
ordinal_encoded = ordinal_enc(X_train, X_valid, ['Sex'])
X_trainR = ordinal_encoded[0]
X_validR = ordinal_encoded[1]

In [45]:
n = set(null_cols(X_trainR))
n

{'Age', 'Cabin', 'Embarked'}

In [62]:
c = set(cat_cols(X_trainR))
c

{'Cabin', 'Embarked', 'Name', 'Ticket'}

In [66]:
c = list(c.union(n))
c

['Name', 'Cabin', 'Ticket', 'Age', 'Embarked']

In [68]:
X_trainR.drop(c, axis=1, inplace=True)
X_validR.drop(c, axis=1, inplace=True)

In [72]:
X_trainR.drop("PassengerId", axis=1, inplace=True)
X_validR.drop("PassengerId", axis=1, inplace=True)

In [76]:
X_trainR.drop("SibSp", axis=1, inplace=True)
X_validR.drop("SibSp", axis=1, inplace=True)

In [77]:
X_trainR.head()

,Pclass,Sex,Parch,Fare
0,3,1.0,0,7.2500
1,1,0.0,0,71.2833
2,3,0.0,0,7.9250
3,1,0.0,0,53.1000
4,3,1.0,0,8.0500


In [78]:
X_validR.head()

,Pclass,Sex,Parch,Fare
0,3,1.0,0,7.8292
1,3,0.0,0,7.0000
2,2,1.0,0,9.6875
3,3,1.0,0,8.6625
4,3,0.0,1,12.2875


In [79]:
score_dataset(X_trainR, X_validR, y_train, y_valid)

0.1869702985794283

In [93]:
predict = predict_RFR(X_trainR, y_train, X_validR)
results = pd.DataFrame(data={
  'PassengerId': test_df['PassengerId'],
  'Survived': predict
})
results = results.map(lambda x: int(round(x)))
results = results.set_index('PassengerId')
results

,Survived
PassengerId,
892,0
893,1
894,0
895,0
896,1
...,...
1305,0
1306,1
1307,0


In [94]:
results.to_csv("./mine.csv")